In [ ]:
# import all needed packages
import sys
sys.path.insert(0, '../')

#matplot
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Markdown, display

#data frame to image
import dataframe_image as dfi

# Datasets
from aif360.datasets import StandardDataset

# Fairness metrics
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric

# Explainers
from aif360.explainers import MetricTextExplainer

# Scalers
from sklearn.preprocessing import StandardScaler

# Classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline

# Bias mitigation techniques
from aif360.algorithms.preprocessing import Reweighing
from aif360.algorithms.inprocessing import PrejudiceRemover

# LIME
from aif360.datasets.lime_encoder import LimeEncoder
import lime
from lime.lime_tabular import LimeTabularExplainer

# seaborn
import seaborn as sns

np.random.seed(1)

In [ ]:
#create datasets
DATA_DIR = '/media/bigdata/10. Stages/3. Afgerond/2020-08 Jesse Kuiper/'

In [ ]:
import pandas as pd
from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

#load data
Dataset14Days = pd.read_csv(DATA_DIR + "Dataset14Days.csv")

In [ ]:
# datasetwhole
Dataset1 = pd.DataFrame(data=Dataset14Days)

In [ ]:
# create corr plot of all variables
corr_mat = Dataset1.corr()

plt.subplots(figsize=(50,50))
sns.heatmap(corr_mat, annot = True)

plt.show()

In [ ]:
# turn the dataset into an AIF360 dataset
def get_favourable1(DoseDiazepamPost):
    return DoseDiazepamPost==0

#dataset = aif360.datasets.StandardDataset(
#    your_pandas_df,
#    label_name,
#    favorable_classes,
#    protected_attribute_names,
#    privileged_classes)

DW = StandardDataset(
    Dataset1,
    "DoseDiazepamPost",
    get_favourable1,
    ["Geslacht"],
    [["Man"]])

In [ ]:
# create LR and RF with unchanged data

# split the date into 3 datasets
(dataset_train,
 dataset_val,
 dataset_test) = DW.split([0.5, 0.8], shuffle=True)

sens_ind = 0
sens_attr = dataset_train.protected_attribute_names[sens_ind]

unprivileged_groups = [{sens_attr: v} for v in
                       dataset_train.unprivileged_protected_attributes[sens_ind]]
privileged_groups = [{sens_attr: v} for v in
                     dataset_train.privileged_protected_attributes[sens_ind]]

In [ ]:
# describe function

def describe(train=None, val=None, test=None):
    if train is not None:
        display(Markdown("#### Training Dataset shape"))
        print(train.features.shape)
    if val is not None:
        display(Markdown("#### Validation Dataset shape"))
        print(val.features.shape)
    display(Markdown("#### Test Dataset shape"))
    print(test.features.shape)
    display(Markdown("#### Favorable and unfavorable labels"))
    print(test.favorable_label, test.unfavorable_label)
    display(Markdown("#### Protected attribute names"))
    print(test.protected_attribute_names)
    display(Markdown("#### Privileged and unprivileged protected attribute values"))
    print(test.privileged_protected_attributes, 
          test.unprivileged_protected_attributes)
    display(Markdown("#### Dataset feature names"))
    print(test.feature_names)

In [ ]:
describe(dataset_train, dataset_val, dataset_test)

In [ ]:
# explore the disparate impact
metric_train = BinaryLabelDatasetMetric(
        dataset_train,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
explainer_train = MetricTextExplainer(metric_train)

print(explainer_train.disparate_impact())

#error is probably due to the test data

### logistic regression

In [ ]:
dataset = dataset_train
model = make_pipeline(StandardScaler(),
                      LogisticRegression(solver='liblinear', random_state=1))
fit_params = {'logisticregression__sample_weight': dataset.instance_weights}

lr_dataset = model.fit(dataset.features, dataset.labels.ravel(), **fit_params)

In [ ]:
from collections import defaultdict

def test(dataset, model, thresh_arr):
    try:
        # sklearn classifier
        y_val_pred_prob = model.predict_proba(dataset.features)
        pos_ind = np.where(model.classes_ == dataset.favorable_label)[0][0]
    except AttributeError:
        # aif360 inprocessing algorithm
        print('aif360 inprocessing algorithm')
        y_val_pred_prob = model.predict(dataset).scores
        pos_ind = 0
    
    metric_arrs = defaultdict(list)
    for thresh in thresh_arr:
        y_val_pred = (y_val_pred_prob[:, pos_ind] > thresh).astype(np.float64)

        dataset_pred = dataset.copy()
        dataset_pred.labels = y_val_pred
        metric = ClassificationMetric(
                dataset, dataset_pred,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)

        
        # calculate the F1 score
        
        metric_arrs['bal_acc'].append((metric.true_positive_rate()
                                     + metric.true_negative_rate()) / 2)
        metric_arrs['F1_score'].append(metric.true_positive_rate() /
                                      (metric.true_positive_rate() + (0.5 *(metric.false_positive_rate() + metric.false_negative_rate())))) 
        
        metric_arrs['FP Diff'].append(metric.false_positive_rate_difference())
                                        
        metric_arrs['disp_imp'].append(metric.disparate_impact())
        metric_arrs['avg_odds_diff'].append(metric.average_odds_difference())
        metric_arrs['stat_par_diff'].append(metric.statistical_parity_difference())
        metric_arrs['eq_opp_diff'].append(metric.equal_opportunity_difference())
        #metric_arrs['theil_ind'].append(metric.theil_index())
    
    return metric_arrs

In [ ]:
thresh_arr = np.linspace(0.1, 0.9, 50)
val_metrics = test(dataset=dataset_val,
                   model=lr_dataset,
                   thresh_arr=thresh_arr)
lr_orig_best_ind = np.argmax(val_metrics['bal_acc'])

In [ ]:
def plot(x, x_name, y_left, y_left_name, y_right, y_right_name, filename=None):
    fig, ax1 = plt.subplots(figsize=(10,7))
    ax1.plot(x, y_left)
    ax1.set_xlabel(x_name, fontsize=16, fontweight='bold')
    ax1.set_ylabel(y_left_name, color='b', fontsize=16, fontweight='bold')
    ax1.xaxis.set_tick_params(labelsize=14)
    ax1.yaxis.set_tick_params(labelsize=14)
    ax1.set_ylim(0.5, 1)

    ax2 = ax1.twinx()
    ax2.plot(x, y_right, color='r')
    ax2.set_ylabel(y_right_name, color='r', fontsize=16, fontweight='bold')
    if 'DI' in y_right_name:
        ax2.set_ylim(0., 1)
    else:
        ax2.set_ylim(-0.25, 1)

    best_ind = np.argmax(y_left)
    ax2.axvline(np.array(x)[best_ind], color='k', linestyle=':')
    ax2.yaxis.set_tick_params(labelsize=14)
    ax2.grid(True)
    if filename:
        fig.savefig(filename)

In [ ]:
disp_imp = np.array(val_metrics['disp_imp'])
disp_imp_err = 1 - np.minimum(disp_imp, 1/disp_imp)
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     disp_imp_err, '1 - min(DI, 1/DI)',
    filename='LR_DI_none.eps')

In [ ]:
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     val_metrics['avg_odds_diff'], 'avg. odds diff.',
    filename='LR_AOD_none.eps')

In [ ]:
def describe_metrics(metrics, thresh_arr):
    best_ind = np.argmax(metrics['bal_acc'])
    print("Threshold corresponding to Best balanced accuracy: {:6.4f}".format(thresh_arr[best_ind]))
    print("Best balanced accuracy: {:6.4f}".format(metrics['bal_acc'][best_ind]))
#     disp_imp_at_best_ind = np.abs(1 - np.array(metrics['disp_imp']))[best_ind]
    disp_imp_at_best_ind = 1 - min(metrics['disp_imp'][best_ind], 1/metrics['disp_imp'][best_ind])
    print("F1 score: {:6.4f}".format(metrics['F1_score'][best_ind]))
    print("FP Diff: {:6.4f}".format(metrics['FP Diff'][best_ind]))
    print("Corresponding 1-min(DI, 1/DI) value: {:6.4f}".format(disp_imp_at_best_ind))
    print("Corresponding average odds difference value: {:6.4f}".format(metrics['avg_odds_diff'][best_ind]))
    print("Corresponding statistical parity difference value: {:6.4f}".format(metrics['stat_par_diff'][best_ind]))
    print("Corresponding equal opportunity difference value: {:6.4f}".format(metrics['eq_opp_diff'][best_ind]))
    print('Corresponding disparate impact value: {:6.4f}'.format(metrics['disp_imp'][best_ind]))
    #print("Corresponding Theil index value: {:6.4f}".format(metrics['theil_ind'][best_ind]))

In [ ]:
describe_metrics(val_metrics, thresh_arr)

In [ ]:
#testing with normal data
lr_orig_metrics = test(dataset=dataset_test,
                       model=lr_dataset,
                       thresh_arr=[thresh_arr[lr_orig_best_ind]])

In [ ]:
describe_metrics(lr_orig_metrics, [thresh_arr[lr_orig_best_ind]])

In [ ]:
# random forrest on original data

dataset = dataset_train
model = make_pipeline(StandardScaler(),
                      RandomForestClassifier(n_estimators=500, min_samples_leaf=25))
fit_params = {'randomforestclassifier__sample_weight': dataset.instance_weights}
rf_orig_panel19 = model.fit(dataset.features, dataset.labels.ravel(), **fit_params)

In [ ]:
# RF validating
thresh_arr = np.linspace(0.1, 0.8, 50)
val_metrics = test(dataset=dataset_val,
                   model=rf_orig_panel19,
                   thresh_arr=thresh_arr)
rf_orig_best_ind = np.argmax(val_metrics['bal_acc'])

In [ ]:
disp_imp = np.array(val_metrics['disp_imp'])
disp_imp_err = 1 - np.minimum(disp_imp, 1/disp_imp)
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     disp_imp_err, '1 - min(DI, 1/DI)',
    filename='RF_DI_none.eps')

In [ ]:
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     val_metrics['avg_odds_diff'], 'avg. odds diff.',
    filename='RF_AOD_none.eps')

In [ ]:
describe_metrics(val_metrics, thresh_arr)

In [ ]:
#Rf model
rf_orig_metrics = test(dataset=dataset_test,
                       model=rf_orig_panel19,
                       thresh_arr=[thresh_arr[rf_orig_best_ind]])

In [ ]:
describe_metrics(rf_orig_metrics, [thresh_arr[rf_orig_best_ind]])

In [ ]:
# transforming data by reweighing

RW = Reweighing(unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups)
dataset_transf_train = RW.fit_transform(dataset_train)

In [ ]:
metric_transf_train = BinaryLabelDatasetMetric(
        dataset_transf_train,
        unprivileged_groups=unprivileged_groups,
        privileged_groups=privileged_groups)
explainer_transf_train = MetricTextExplainer(metric_transf_train)

print(explainer_transf_train.disparate_impact())

In [ ]:
#training LR on reweight data
dataset = dataset_transf_train
model = make_pipeline(StandardScaler(),
                      LogisticRegression(solver='liblinear', random_state=1))
fit_params = {'logisticregression__sample_weight': dataset.instance_weights}
lr_transf = model.fit(dataset.features, dataset.labels.ravel(), **fit_params)

In [ ]:
#validation
thresh_arr = np.linspace(0.01, 0.8, 50)
val_metrics = test(dataset=dataset_val,
                   model=lr_transf,
                   thresh_arr=thresh_arr)
lr_transf_best_ind = np.argmax(val_metrics['bal_acc'])

In [ ]:
disp_imp = np.array(val_metrics['disp_imp'])
disp_imp_err = 1 - np.minimum(disp_imp, 1/disp_imp)
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     disp_imp_err, '1 - min(DI, 1/DI)',
    filename='LR_DI_reweigh.eps')

In [ ]:
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     val_metrics['avg_odds_diff'], 'avg. odds diff.',
    filename='LR_AOD_reweigh.eps')

In [ ]:
describe_metrics(val_metrics, thresh_arr)

In [ ]:
lr_transf_metrics = test(dataset=dataset_test,
                         model=lr_transf,
                         thresh_arr=[thresh_arr[lr_transf_best_ind]])

In [ ]:
describe_metrics(lr_transf_metrics, [thresh_arr[lr_transf_best_ind]])

In [ ]:
#training RF on reweight data

dataset = dataset_transf_train
model = make_pipeline(StandardScaler(),
                      RandomForestClassifier(n_estimators=500, min_samples_leaf=25))
fit_params = {'randomforestclassifier__sample_weight': dataset.instance_weights}
rf_transf = model.fit(dataset.features, dataset.labels.ravel(), **fit_params)

In [ ]:
thresh_arr = np.linspace(0.01, 0.8, 50)
val_metrics = test(dataset=dataset_val,
                   model=rf_transf,
                   thresh_arr=thresh_arr)
rf_transf_best_ind = np.argmax(val_metrics['bal_acc'])

In [ ]:
disp_imp = np.array(val_metrics['disp_imp'])
disp_imp_err = 1 - np.minimum(disp_imp, 1/disp_imp)
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     disp_imp_err, '1 - min(DI, 1/DI)',
    filename='RF_DI_reweigh.eps')

In [ ]:
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     val_metrics['avg_odds_diff'], 'avg. odds diff.',
    filename='RF_AOD_reweigh.eps')

In [ ]:
describe_metrics(val_metrics, thresh_arr)

In [ ]:
rf_transf_metrics = test(dataset=dataset_test,
                         model=rf_transf,
                         thresh_arr=[thresh_arr[rf_transf_best_ind]])

In [ ]:
describe_metrics(rf_transf_metrics, [thresh_arr[rf_transf_best_ind]])

In [ ]:
# inprocess mitigation - prejudice remover
model = PrejudiceRemover(sensitive_attr=sens_attr, eta=25.0)
pr_orig_scaler = StandardScaler()

dataset = dataset_train.copy()
dataset.features = pr_orig_scaler.fit_transform(dataset.features)

pr_orig_DW = model.fit(dataset)

In [ ]:
thresh_arr = np.linspace(0.01, 0.8, 50)

dataset = dataset_val.copy()
dataset.features = pr_orig_scaler.transform(dataset.features)

val_metrics = test(dataset=dataset,
                   model=pr_orig_DW,
                   thresh_arr=thresh_arr)
pr_orig_best_ind = np.argmax(val_metrics['bal_acc'])

In [ ]:
disp_imp = np.array(val_metrics['disp_imp'])
disp_imp_err = 1 - np.minimum(disp_imp, 1/disp_imp)
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     disp_imp_err, '1 - min(DI, 1/DI)',
    filename='LR_DI_PR.eps')

In [ ]:
plot(thresh_arr, 'Classification Thresholds',
     val_metrics['bal_acc'], 'Balanced Accuracy',
     val_metrics['avg_odds_diff'], 'avg. odds diff.',
    filename='LR_AOD_PR.eps')

In [ ]:
describe_metrics(val_metrics, thresh_arr)

In [ ]:
dataset = dataset_test.copy()
dataset.features = pr_orig_scaler.transform(dataset.features)

pr_orig_metrics = test(dataset=dataset,
                       model=pr_orig_DW,
                       thresh_arr=[thresh_arr[pr_orig_best_ind]])

In [ ]:
describe_metrics(pr_orig_metrics, [thresh_arr[pr_orig_best_ind]])

In [ ]:
#table summary of the results
pd.set_option('display.multi_sparse', False)
results = [lr_orig_metrics, rf_orig_metrics, lr_transf_metrics,
           rf_transf_metrics, pr_orig_metrics]
debias = pd.Series(['']*2 + ['Reweighing']*2
                 + ['Prejudice Remover'],
                   name='Bias Mitigator')
clf = pd.Series(['Logistic Regression', 'Random Forest']*2 + [''],
                name='Classifier')
results_df = pd.concat([pd.DataFrame(metrics) for metrics in results], axis=0).set_index([debias, clf])
results_df

I looked at this by eye, and it's the same as was published in Jesse's thesis

In [ ]:
results_df.rename(columns={'bal_acc': 'Bal acc', 'F1_score': 'F1 score', 'disp_imp': 'Disp imp', 
                           'avg_odds_diff': 'Avg odds diff', 'stat_par_diff': 'Stat par diff', 
                           'eq_opp_diff': 'Eq opp diff'}, inplace=True)
results_df.drop(columns=['FP Diff'], inplace=True)

In [ ]:
results_df.reset_index(inplace=True)
results_df.fillna('')

In [ ]:
cols = results_df.columns
print(' & '.join(cols) + ' \\\\')
print('\\hline')
print('\\hline')
for elll in [' & '.join(['{:.3f}'.format(ell) if type(ell) != str else ell for ell in row]) + ' \\\\' for row in results_df[[el for el in cols]].values]:
    print(elll)